In [ ]:
# default_exp models.game_data

In [ ]:
from nbdev.showdoc import *

In [ ]:
#exporti

import pandas as pd
import statsapi as mlb
from pydantic import (
    BaseModel,
    Field,
    validator,
    conint,
    constr,
    confloat,
    root_validator
)
from MLB_DataDevTools.models.base_models import *
from enum import Enum
import datetime as dt
from typing import Optional,Dict

from MLB_DataDevTools.mlb_api import get_season_game_pks
from MLB_DataDevTools.database import create_mlb_engine


In [ ]:
#slow 

engine = create_mlb_engine()
games = pd.read_sql_table('game',engine.connect())
games.columns

In [ ]:
gamePks = get_season_game_pks(2019)

In [ ]:
import random

random_ix = random.randint(0,len(gamePks)-1)

example_game = mlb.get('game',{'gamePk':gamePks[random_ix]})
print(example_game['gamePk'])

print(example_game.keys())

gameData = example_game['gameData']

## Game Details

In [ ]:
#exporti 

from MLB_DataDevTools.models.base_models import GamedayType,GameType

class GameDetails(BaseModel):
    pk: int
    type: GameType
    doubleHeader: bool
    id: str
    gamedayType: GamedayType
    tiebreaker: bool
    gameNumber: conint(gt=0,lt=3)
    calendarEventID: str
    season: str
    seasonDisplay: str
    
    @validator('doubleHeader')
    def validate_double_header(cls,v):
        if v == 'S':
            return True
        else:
            return v

In [ ]:
show_doc(GameDetails)

In [ ]:
gameData['game']

In [ ]:
print(
    GameDetails(**gameData['game']).json(indent=2)
)

## Game Datetime

In [ ]:
gameData['datetime']

In [ ]:
#exporti 

class DayNight(str,Enum):
    day='day'
    night='night'
class AmPm(str,Enum):
    AM='AM'
    PM='PM'

In [ ]:
#exporti

class GameDatetime(BaseModel):
    dateTime: dt.datetime = Field(...,description="Timezone unaware datetime for the game")
    originalDate: dt.date
    dayNight: DayNight
    time: dt.time = Field(...,description='Local time for the game')
    ampm: AmPm
    
    @root_validator
    def validate_game_time(cls,values):
        """
        Adds 12 hours to the game time if 'pm'
        """
        if values['ampm']=='PM':
            values['time'] = dt.time(
                values['time'].hour + 12,
                values['time'].minute
            )
            return values
        return values

In [ ]:
gameData['datetime']

In [ ]:
show_doc(GameDatetime)

In [ ]:
GameDatetime.schema()

In [ ]:
print(GameDatetime(**gameData['datetime']).json(indent=2))

## Game Status

In [ ]:
gameData['status']

In [ ]:
game_statuses = mlb.get('meta',{'type':'gameStatus'})
game_statuses[0]

In [ ]:
game_status_values = {
    k:set(x.get(k) for x in game_statuses)
    for k in gameData['status'].keys()
}

In [ ]:
game_status_values['abstractGameState']

In [ ]:
game_status_values['abstractGameCode']

In [ ]:
#exporti
from MLB_DataDevTools.models.base_models import AbstractGameCode,AbstractGameState

class GameStatus(BaseModel):
    abstractGameState: AbstractGameState
    codedGameState: constr(max_length=2)
    detailedState: str
    statusCode: constr(max_length=2)
    reason: Optional[str] = None 
    startTimeTBD: Optional[bool] = False
    abstractGameCode: AbstractGameCode

In [ ]:
show_doc(GameStatus)

In [ ]:
gameData['status']

In [ ]:
print(GameStatus(**gameData['status']).json(indent=2))

## Team

In [ ]:
gameData['teams'].keys()

In [ ]:
team = gameData['teams']['home']
{k: list(v.keys()) for k,v in team.items() if type(v)==dict}

### Venue

Some of the values here are abridged versions of what the MLB API would return if called directly. For example, the `venue` enpoint returns name and id, but the springVenue value here just provides the id. 

In [ ]:
mlb.get('venue',{'venueIds':'4705'})

In [ ]:
team['venue']

In [ ]:
team['springVenue']

In [ ]:
mlb.get('venue',{'venueIds':'4705'})

In [ ]:
#exporti 

class TeamVenueBase(MLBEndpointBase):
    id: int
class TeamVenue(TeamVenueBase):
    name: str

In [ ]:
print(TeamVenueBase(**team['springVenue']).json(indent=2))

In [ ]:
print(TeamVenue(**team['venue']).json())

### Record

In [ ]:
team['record']

In [ ]:
#exporti 

class TeamRecord(BaseModel):
    wins: conint(ge=0)
    losses: conint(ge=0)
    winningPercentage: confloat(ge=0,lt=1) = Field(...,alias='pct')
    

class GameTeamRecord(TeamRecord):
    gamesPlayed: int
    wildCardGamesBack: CustomInt
    leagueGamesBack: CustomInt
    springLeagueGamesBack: CustomInt
    sportGamesBack: CustomInt
    leagueRecord: TeamRecord
    divisionLeader: bool
    records: dict # not sure what's supposed to go here... 

    class Config:
        allow_population_by_field_name=True

In [ ]:
team['record']['leagueRecord']

In [ ]:
TeamRecord(**team['record']['leagueRecord'])

In [ ]:
team['record']

In [ ]:
print(GameTeamRecord(**team['record'],by_alias=False).json(indent=2))

In [ ]:
team.keys()

In [ ]:
team['sport']

In [ ]:
mlb.get('sports',{'sportId':1})

In [ ]:
team['division']

In [ ]:
import requests 

requests.get(mlb.BASE_URL + 'v1/divisions/204').json()

In [ ]:
#exporti 

class GameTeam(MLBEndpointReference):
    season: int
    venue: TeamVenue
    springVenue: TeamVenueBase
    teamCode: str
    fileCode: str
    abbreviation: str
    teamName: str
    locationName: str
    firstYearOfPlay: str
    league: MLBEndpointReference
    division: MLBEndpointReference
    shortName: str
    record: GameTeamRecord
    springLeague: MLBEndpointReference
    allStarStatus: bool
    active: bool
    
        

In [ ]:
game_team = GameTeam(**team)
print(game_team.json(indent=2))

In [ ]:
game_team.springLeague

In [ ]:
game_team.record

In [ ]:
game_teams = [GameTeam(**gameData['teams'][tm]) for tm in ('home','away')]

## Players

This is the full response from the `players` endpoint as well

In [ ]:
player_example = list(gameData['players'].values())[0]

In [ ]:
mlb.get('people',{'personIds':player_example['id']})['people'][0] == player_example

In [ ]:
player_example

In [ ]:
positions = mlb.get('meta',{'type':'positions'})
positions[0]

In [ ]:
player_example['primaryPosition']

In [ ]:
gameData['primaryDatacaster']

In [ ]:
example_person = mlb.get('people',{'personIds':gameData['primaryDatacaster']['id']})['people'][0]
example_person

In [ ]:
person = MLBPerson(**example_person)

In [ ]:
person

In [ ]:
#exporti 

class Player(MLBPerson):
    primaryNumber: Optional[int] = None
    birthDate: dt.date
    currentAge: int
    birthCity: str
    birthStateProvince: Optional[str] = None
    birthCountry: str

    middleName: Optional[str] = None

    pitchHand: PlayerHandedness
    batSide: PlayerHandedness

    draftYear: Optional[int] = None
    mlbDebutDate: dt.date
    
    strikeZoneTop: float
    strikeZoneBottom: float
        

In [ ]:
player_example

In [ ]:
Player(**player_example)

In [ ]:
players = {k:Player(**v) for k,v in gameData['players'].items()}

## Venue

In [ ]:
venues = pd.read_sql_table('venue',engine.connect())

print(venues['fieldInfo_turfType'].unique())
print(venues['fieldInfo_roofType'].unique())

In [ ]:
gameData['venue']

In [ ]:
#exporti 

class GameVenue(TeamVenue):
    location: VenueLocation
    timeZone: TimeZone
    fieldInfo: FieldInfo

In [ ]:
GameVenue(**gameData['venue'])

## Weather

In [ ]:
gameData['weather']

In [ ]:
mlb.get('meta',{'type':'windDirection'})

In [ ]:
#exporti 

class GameWeather(BaseModel):
    condition: str
    temp: int
    wind: str

In [ ]:
GameWeather(**gameData['weather'])

In [ ]:
gameData['gameInfo']

## Game Info

In [ ]:
#exporti 

class GameInfo(BaseModel):
    attendance: int
    gameDurationMinutes: int
    
    @property
    def game_duration(self):
        return dt.timedelta(minutes=self.gameDurationMinutes)

In [ ]:
game_info = GameInfo(**gameData['gameInfo'])
game_info.game_duration

In [ ]:
duration = game_info.game_duration

## Review

In [ ]:
gameData['review']

In [ ]:
#exporti 

class Challenges(BaseModel):
    used: int
    remaining: int

class GameReview(BaseModel):
    hasChallenges: bool
    away: Challenges
    home: Challenges

In [ ]:
GameReview(**gameData['review'])

## Flags

In [ ]:
gameData['flags']

In [ ]:
#exporti 

class GameFlags(BaseModel):
    noHitter: bool
    perfectGame: bool
    awayTeamNoHitter: bool
    awayTeamPerfectGame: bool
    homeTeamNoHitter: bool
    homeTeamPerfectGame: bool

In [ ]:
GameFlags(**gameData['flags'])

In [ ]:
gameData.keys()

## Probable Pitchers, official Scorer, datacaster

In [ ]:
gameData['probablePitchers']

In [ ]:
[PersonBase(**gameData['probablePitchers'][tm]) for tm in ('home','away')]

In [ ]:
PersonBase(**gameData['officialScorer'])

In [ ]:
PersonBase(**gameData['primaryDatacaster'])

## GameData

In [ ]:
gameData.keys()

In [ ]:
#exporti 

class HomeAway(str,Enum):
    home = 'home'
    away = 'away'

In [ ]:
#export 

class GameData(BaseModel):
    game: GameDetails
    datetime: GameDatetime
    status: GameStatus
    teams: Dict[HomeAway,GameTeam]
    players: Dict[str,Player]
    venue: GameVenue
    gameInfo: GameInfo
    review: GameReview
    flags: GameFlags
    probablePitchers: Dict[HomeAway,PersonBase]
    officialScorer: PersonBase
    primaryDatacaster: PersonBase

In [ ]:
game_data = GameData(**gameData)

In [ ]:
game_data.teams['away']

In [ ]:
GameData.schema()

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()